# Construction of a model for predicting the amount of population per area

first we will upload the basic python library

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import joblib
from matplotlib import pyplot
from numpy import mean
from numpy import std

then we will upload sklearn for statistical-learning for scientific data processing 

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV
from sklearn.model_selection import RepeatedKFold, StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline i

In [3]:
mport make_pipeline
from sklearn.preprocessing import StandardScaler

# create a class 'MyModel' which will hold all the functions

In [ ]:
class MyModel():    

removing buildings with apartment two big or small

In [ ]:
    def area_vs_app_per_lev(self,data):
        # creat ne column with the ratio of ORIG_AREA vs App_Per_Lvl
        data['Area_per_App'] = data.ORIG_AREA/data.App_Per_Lvl
        small_app = data.loc[data.Area_per_App < 40, :]
    
        big_app = data.loc[data.Area_per_App >= 200, :]
        new_data = new_data = data.loc[(data.Area_per_App <= 200) & (data.Area_per_App > 40), :]

visualization of the data using seaborn library

In [ ]:
        sns.pairplot(data)
        sns.pairplot(small_app)
        sns.pairplot(big_app)
        sns.pairplot(new_data)
        small_app = small_app.drop('Area_per_App',axis=1)
        big_app = big_app.drop('Area_per_App',axis=1)
        new_data = new_data.drop('Area_per_App',axis=1)

the function returns all the small apartment(<40), the big apartments (>200) and the data between these values 

In [ ]:
        return small_app, big_app, new_data

creat a function to predict the number of floors in each building

In [ ]:
    def predict_num_floors(self,data):

creat new column of BLDG_Height vs Floors


In [ ]:
        data['height_per_floors'] = data.BLDG_Height/data.Floors

separate the data according to bad or good ratio

In [ ]:
        bad_ratio_height_per_floors = data.loc[((data.height_per_floors < 3) | (data.height_per_floors > 5.5) | ((data.Floors != 1) & ((data.height_per_floors > 4.5) & (data.height_per_floors <= 5.5)))), :]
        good_ratio_height_per_floors = data.loc[( ((data.height_per_floors >= 3) & (data.height_per_floors <= 4.5)) | (data.Floors == 1 & ((data.height_per_floors >= 4.5) & (data.height_per_floors <= 5.5)))), :]
        no_floors = data.loc[np.isnan(data.Floors), :]
        no_BLDG_Height = data.loc[np.isnan(data.BLDG_Height), :]
        bad_ratio_height_per_floors_2 = pd.concat([bad_ratio_height_per_floors,no_floors],ignore_index=True)

visualization of the data using seaborn library

In [ ]:
        sns.pairplot(bad_ratio_height_per_floors)
        sns.pairplot(good_ratio_height_per_floors)

creat a model to predict number of floors in a building

In [ ]:
        x_train = good_ratio_height_per_floors['BLDG_Height']
        y_train = good_ratio_height_per_floors['Floors']
        x_test = bad_ratio_height_per_floors_2['BLDG_Height']

train the model

In [ ]:
        model = LinearRegression().fit(np.asarray(x_train).reshape(len(x_train), 1), y_train)
        score_model_train = model.score(np.asarray(x).reshape(len(x), 1), y_train)

get prediction

In [ ]:
        predict_floor = model.predict(np.asarray(x_test).reshape(len(x_test), 1))

round the numbers of predictions

In [ ]:
        round_predict_floor = np.rint(predict_floor)

replace the predictions with the old 'Floors' column

In [ ]:
        bad_ratio_height_per_floors_2 = bad_ratio_height_per_floors_2.drop(['Floors','height_per_floors'], axis=1)
        bad_ratio_height_per_floors_2['Floors'] = round_predict_floor
        good_ratio_height_per_floors = good_ratio_height_per_floors.drop(['height_per_floors'], axis=1)

merge the two data sets into one final data

In [ ]:
        final_data = pd.concat([bad_ratio_height_per_floors_2, good_ratio_height_per_floors], ignore_index=True)

        return final_data,no_BLDG_Height

   

create a function for removing of old buildings
    

In [ ]:
    def old_building(self,data):
        # check for missing values (building without construction year)
        missing_values=data.isnull().sum()
        missing_year = data.loc[np.isnan(data.Construction_Year), :]

        old_build = data.loc[data.Construction_Year < 1950, :]
        old_build_2 = pd.concat([old_build,missing_year], ignore_index=True)
        new_build = data.loc[data.Construction_Year >= 1950, :]

        return old_build_2, new_build

use skew metud to smoot values in the data with log


In [ ]:
    def skew_data(self,data):
        # skew = data.skew()
        new_data = data.copy()
        # We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
        if isinstance(data, pd.DataFrame):
            for column in new_data.columns:
                new_data[column] = np.log1p(np.asarray(new_data[column]))
        else:
            new_data = np.log1p(np.asarray(new_data))
        return new_data

    

creat a function to get a list of models to evaluate


In [ ]:
    def get_models(self):
        models = dict()
        models['lr'] = LinearRegression()
        models['knn'] = KNeighborsRegressor()
        models['rfr'] = RandomForestRegressor()
        models['abr'] = AdaBoostRegressor()
        models['gbr'] = GradientBoostingRegressor()
        models['svm'] = make_pipeline(StandardScaler(), SVR())
        models['stacking'] = obj1.get_stacking()
        return models

evaluate a given model using cross-validation


In [ ]:
    def evaluate_model(self,model,x,y):
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
        return scores

evaluate the prediction of given model accuracy

In [ ]:
    def evaluate_accuracy(self,y_true, y_pred):
        l = len(y_pred)
        match = 0
        match_minus_1 = 0
        match_plus_1 = 0
        for i, j in zip(y_true,y_pred):
            if i == j:
                match += 1
            elif i == j+1:
                match_plus_1+=1
            elif i == j-1:
                match_minus_1+=1
        return match/l, match_plus_1/l, match_minus_1/l

create a function to get a stacking ensemble of models

In [ ]:
    def get_stacking(self):
        # define the base models
        level0 = list()
        level0.append(('knn', KNeighborsRegressor()))
        level0.append(('rfr', RandomForestRegressor()))
        level0.append(('abr', AdaBoostRegressor()))
        level0.append(('gbr', GradientBoostingRegressor()))
        level0.append(('svm', make_pipeline(StandardScaler(), SVR())))
        # define meta learner model
        level1 = LinearRegression()
        # define the stacking ensemble
        model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
        return model

create a function to avaluate best parameters for every model

In [ ]:
    def best_params(self, x, y, model, params):
        gd_sr = GridSearchCV(estimator=model,param_grid=params,scoring='neg_mean_absolute_error',cv=5,n_jobs=-1)
        gd_sr.fit(x,y)
        return gd_sr.best_params_,gd_sr.best_estimator_, gd_sr.best_score_

# upload the data

In [ ]:
if __name__ == '__main__':

In [ ]:
    file_path = r'\BLDG_2020.xlsx'
    all_data = pd.read_excel(file_path,sheet_name='Sheet1')
    all_data_correct = pd.read_excel(file_path,sheet_name='Sheet2')

visualization of the data using seaborn library

In [ ]:
    sns.pairplot(all_data_correct)

create the class MyModel where all functions is in

In [ ]:
    obj1=MyModel()

# preprocessing stage

In [ ]:
    small_ap,big_ap,all_data_for_model = obj1.area_vs_app_per_lev(all_data_correct)
    all_data_for_model_2 = obj1.predict_num_floors(all_data_for_model)
    old_buildings, all_data_for_model_3 = obj1.old_building(all_data_for_model_2)
    remaining_data = pd.concat([small_ap, big_ap, old_buildings], ignore_index=True)

Check for duplicate values in Pandas dataframe column

In [ ]:
    double_in_model = all_data_for_model_3['UNIQ_ID'].duplicated().any()
    double_in_remainig = remaining_data['UNIQ_ID'].duplicated().any()

save the final data to reduce time

In [ ]:
    all_data_for_model_3.to_csv(path_or_buf=r'\model_data.csv', index=False)
    remaining_data.to_csv(path_or_buf=r'\remaining_data.csv', index=False)

upload the data for the model after pre processing

In [ ]:
    file_path_2 = r'\model_data.csv'
    all_data_for_model_4 = pd.read_csv(file_path_2)

# creat x and y for the model

In [ ]:
    all_data_x = pd.DataFrame(all_data_for_model_4, columns=['ORIG_AREA','BLDG_Height','Floors'])
    all_data_y = all_data_for_model_4['App_Per_Lvl']

split the data to train and test

In [ ]:
    x_train, x_test, y_train, y_test = train_test_split(all_data_x, all_data_y, test_size=0.2)

skew data using np.log

In [ ]:
    x_train_skew = obj1.skew_data(x_train)
    x_test_skew = obj1.skew_data(x_test)
    y_test_skew = obj1.skew_data(y_test)
    y_train_skew = obj1.skew_data(y_train)

# get the models evaluations 

In [ ]:
    models = obj1.get_models()
    # evaluate the models and store results
    results, names = list(), list()
    for name, model in models.items():
        scores = obj1.evaluate_model(model,x_train,y_train)
        results.append(scores)
        names.append(name)
        print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
    # plot model performance for comparison
    pyplot.boxplot(results, labels=names, showmeans=True)
    pyplot.show()

   #  train the model

In [ ]:
    trained_meta_model = obj1.get_stacking()
    trained_meta_model.fit(x_train_skew,y_train_skew)
    predicted_y = np.rint(np.expm1(trained_meta_model.predict(x_test_skew)))
    acc, acc_plus,acc_minus = obj1.evaluate_accuracy(np.asarray(y_test.tolist()),predicted_y)
    evaluate_data = pd.DataFrame(x_test)
    evaluate_data['true_y'] = y_test
    evaluate_data['pre_y'] = predicted_y

save the model to disk

In [ ]:
    filename = r'\finalized_model.sav'
    joblib.dump(trained_meta_model, filename)

load the model from the disk

In [ ]:
    loaded_model = joblib.load(filename)

creat x and y for the prediction

In [ ]:
    list_of_id = all_data_for_model_4['UNIQ_ID'].tolist()
    all_data_2, no_BLDG_Height = obj1.predict_num_floors(all_data)
    data_to_predict = all_data_2[~all_data_2.UNIQ_ID.isin(list_of_id)]
    data_to_predict_x = pd.DataFrame(data_to_predict, columns=['ORIG_AREA','BLDG_Height','Floors'])

    # skew data using np.log
    data_to_predict_x_skew = obj1.skew_data(data_to_predict_x)

get the final prediction of apartment for area

In [ ]:
    final_prediction = np.expm1(loaded_model.predict(data_to_predict_x_skew))
    final_round_prediction = np.rint(final_prediction)
    data_to_predict = data_to_predict.drop(['App_Per_Lvl','AppCount'],axis=1)

add prediction to the original data

In [ ]:
    data_to_predict['App_Per_Lvl'] = final_prediction
    data_to_predict['App_Per_Lvl_round'] = final_round_prediction

    data_to_predict['AppCount'] = data_to_predict.Floors * data_to_predict.App_Per_Lvl
    data_to_predict['AppCount_round'] = data_to_predict.Floors * data_to_predict.App_Per_Lvl_round


In [ ]:
    all_data_3 = pd.concat([data_to_predict,all_data_for_model_4],ignore_index=True)
    double_in_model = all_data_3['UNIQ_ID'].duplicated().any()
    all_data_3.to_csv(path_or_buf=r'\final_data.csv', index=False)

# Grid Search cv 

I used GridSearchCV to better tuning the parameters for every model i used.

In [ ]:
    # using Grid Search cv for better accuracy
    knn_params = {'n_neighbors': [3, 4, 5, 6, 7, 8]}
    gradient_boosting_params = {'max_depth':[3, 5 ,7],'min_samples_leaf': [1, 3, 5, 7],'learning_rate': [0.01, 0.1, 1, 10],'n_estimators': [100, 300, 500, 800]}
    random_forest_param = {'n_estimators': [50, 100, 300, 500 ],'criterion': ['mse', 'mae'],'bootstrap': [True, False],'max_depth': [2, 4, 6, 8,10], 'min_samples_leaf': [1, 3, 5, 7]}
    adaboost_param = {'learning_rate': [0.01, 0.1, 1, 10],'n_estimators': [50,100, 300, 500]}
    svr_param = {'C': [1, 10, 100],'tol': [1e-3,1e-4, 1e-5],'epsilon': [0.01, 0.1, 1, 10]}

    knn_best_params,knn_best_estimator,knn_best_score = obj1.best_params(x_train,y_train,KNeighborsRegressor(),knn_params)
    print(knn_best_params)
#     best pamaters for KNeighborsRegressor is -{'n_neighbors': 8}

    gradient_boosting_best_params,gradient_boosting_best_estimator,gradient_boosting_best_score = obj1.best_params(x_train, y_train, GradientBoostingRegressor(),gradient_boosting_params)
    print(gradient_boosting_best_params)
#     best pamaters 2 for GradientBoostingRegressor is

    random_forest_best_params,random_forest_best_estimator,random_forest_best_score = obj1.best_params(x_train, y_train, RandomForestRegressor(), random_forest_param)
    print(random_forest_best_params)
#     best pamaters  for RandomForestRegressor is

    adaboost_best_params,adaboost_best_estimator,adaboost_best_score = obj1.best_params(x_train, y_train, AdaBoostRegressor(), adaboost_param)
    print(adaboost_best_params)
#     best pamaters  for AdaBoostRegressor is {'learning_rate': 0.01, 'n_estimators': 100}

    svr_best_params,svr_best_estimator,svr_best_score = obj1.best_params(x_train, y_train, make_pipeline(StandardScaler(), SVR()), svr_param)
    print(svr_best_params)
#     best pamaters  for SVR is